In [3]:
%%writefile /Users/ahmad/Desktop/CompVision/02-seam-carve/seam_carve.py
# %load /Users/ahmad/Desktop/02-seam-carve/seam_carve.py
import numpy as np
from numpy import zeros
 
def gradi(arr):
    gradx = zeros(arr.shape)
    grady = zeros(arr.shape)
    for i in range(0, arr.shape[0]):
        for j in range(0, arr.shape[1]):
            if (i == 0):
                gradx[i][j] = arr[i + 1][j] - arr[i][j]
            elif (i == (arr.shape[0] - 1)):
                gradx[i][j] = arr[i][j] - arr[i - 1][j]
            else:
                gradx[i][j] = arr[i + 1][j] - arr[i - 1][j]
            if (j == 0):
                grady[i][j] = arr[i][j + 1] - arr[i][j]
            elif (j == (arr.shape[1] - 1)):
                grady[i][j] = arr[i][j] - arr[i][j - 1]
            else:
                grady[i][j] = arr[i][j + 1] - arr[i][j - 1]
    return (gradx ** 2 + grady ** 2) ** 0.5
 
def seam_carve(img, mode, mask=None):
    # dummy implementation — delete rightmost column of image
    resized_img = img[:, :-1, :]
    gray = 0.299 * img[..., 0]
    gray[:] += 0.587 * img[..., 1]
    gray[:] += 0.114 * img[..., 2]
    import numpy as np
    arr = gray
#         grad = gradi(arr)
    gradx, grady = np.gradient(arr)
    gradx *= 2
    grady *= 2
    for i in range(gradx.shape[1]):
        gradx[0, i] = (gradx[0, i] / 2)
        gradx[gradx.shape[0] - 1, i] = (gradx[gradx.shape[0] - 1, i] / 2)
    for i in range(gradx.shape[0]):
        grady[i, 0] = (grady[i, 0] / 2)
        grady[i, gradx.shape[1] - 1] = (grady[i, gradx.shape[1] - 1] / 2)
    grad = np.sqrt(np.square(gradx) + np.square(grady))
    seamMatrix = zeros((grad.shape[0], grad.shape[1], 2))
    if mask is None:
        resized_mask = None
    else:
        masker = 256 * seamMatrix.shape[0] * seamMatrix.shape[1]
        resized_mask = mask[:, :-1]
        grad += masker * mask
    carve_mask = zeros((img.shape[0], img.shape[1]))
    if ((mode == 'horizontal shrink') or (mode == 'horizontal expand')):
        for k in range(0, grad.shape[1]):
            seamMatrix[0][k][0] = grad[0][k]
            seamMatrix[0][k][1] = 0
        for i in range(1, grad.shape[0]):
            for j in range(0, grad.shape[1]):
                if (j == 0):
                    if (seamMatrix[i - 1, j, 0] <= seamMatrix[i - 1, j + 1, 0]):
                        seamMatrix[i, j, 0] = grad[i, j] + seamMatrix[i - 1, j, 0]
                        seamMatrix[i, j, 1] = j
                    else:
                        seamMatrix[i, j, 0] = grad[i, j] + seamMatrix[i - 1, j + 1, 0]
                        seamMatrix[i, j, 1] = j + 1
                elif (j == (grad.shape[1] - 1)):
                    if (seamMatrix[i - 1, j - 1, 0] <= seamMatrix[i - 1, j, 0]):
                        seamMatrix[i, j, 0] = grad[i, j] + seamMatrix[i - 1, j - 1, 0]
                        seamMatrix[i, j, 1] = j - 1
                    else:
                        seamMatrix[i, j, 0] = grad[i, j] + seamMatrix[i - 1, j, 0]
                        seamMatrix[i, j, 1] = j
                else:
                    if (seamMatrix[i - 1, j - 1, 0] <= seamMatrix[i - 1, j, 0]):
                        if (seamMatrix[i - 1, j - 1, 0] <= seamMatrix[i - 1, j + 1, 0]):
                            seamMatrix[i, j, 0] = grad[i, j] + seamMatrix[i - 1, j - 1, 0]
                            seamMatrix[i, j, 1] = j - 1
                        else:
                            seamMatrix[i, j, 0] = grad[i, j] + seamMatrix[i - 1, j + 1, 0]
                            seamMatrix[i, j, 1] = j + 1
                    else:
                        if (seamMatrix[i - 1, j, 0] <= seamMatrix[i - 1, j + 1, 0]):
                            seamMatrix[i, j, 0] = grad[i, j] + seamMatrix[i - 1, j, 0]
                            seamMatrix[i, j, 1] = j
                        else:
                            seamMatrix[i, j, 0] = grad[i, j] + seamMatrix[i - 1, j + 1, 0]
                            seamMatrix[i, j, 1] = j + 1
        indexLast = np.argmin(seamMatrix[(grad.shape[0] - 1), :, 0])
        indexLast = int(round(indexLast))
        for t in range(0, grad.shape[0]):
            carve_mask[grad.shape[0] - 1 - t][indexLast] = 1
            indexLast = seamMatrix[grad.shape[0] - 1 - t][indexLast][1]
            indexLast = int(round(indexLast))
    else:
        for k in range(0, grad.shape[0]):
            seamMatrix[k][0][0] = grad[k][0]
            seamMatrix[k][0][1] = 0
        for j in range(1, grad.shape[1]):
            for i in range(0, grad.shape[0]):
                if (i == 0):
                    if (seamMatrix[i , j - 1, 0] <= seamMatrix[i + 1, j - 1, 0]):
                        seamMatrix[i, j, 0] = grad[i, j] + seamMatrix[i, j - 1, 0]
                        seamMatrix[i, j, 1] = i
                    else:
                        seamMatrix[i, j, 0] = grad[i, j] + seamMatrix[i + 1, j - 1, 0]
                        seamMatrix[i, j, 1] = i + 1
                elif (i == (grad.shape[0] - 1)):
                    if (seamMatrix[i - 1, j - 1, 0] <= seamMatrix[i, j - 1, 0]):
                        seamMatrix[i, j, 0] = grad[i, j] + seamMatrix[i - 1, j - 1, 0]
                        seamMatrix[i, j, 1] = i - 1
                    else:
                        seamMatrix[i, j, 0] = grad[i, j] + seamMatrix[i, j - 1, 0]
                        seamMatrix[i, j, 1] = i
                else:
                    if (seamMatrix[i - 1, j - 1, 0] <= seamMatrix[i, j - 1, 0]):
                        if (seamMatrix[i - 1, j - 1, 0] <= seamMatrix[i + 1, j - 1, 0]):
                            seamMatrix[i, j, 0] = grad[i, j] + seamMatrix[i - 1, j - 1, 0]
                            seamMatrix[i, j, 1] = i - 1
                        else:
                            seamMatrix[i, j, 0] = grad[i, j] + seamMatrix[i + 1, j - 1, 0]
                            seamMatrix[i, j, 1] = i + 1
                    else:
                        if (seamMatrix[i , j - 1, 0] <= seamMatrix[i + 1, j - 1, 0]):
                            seamMatrix[i, j, 0] = grad[i, j] + seamMatrix[i, j - 1, 0]
                            seamMatrix[i, j, 1] = i
                        else:
                            seamMatrix[i, j, 0] = grad[i, j] + seamMatrix[i + 1, j - 1, 0]
                            seamMatrix[i, j, 1] = i + 1
#                         tempArr = (seamMatrix[i - 1][j - 1][0], seamMatrix[i][j - 1][0], seamMatrix[i + 1][j - 1][0])
#                         seamMatrix[i][j][0] = min(tempArr) + grad[i][j]
#                         seamMatrix[i][j][1] = (tempArr.index(min(tempArr))) + i - 1
        indexLast = np.argmin(seamMatrix[:, grad.shape[1] - 1, 0])
        indexLast = int(round(indexLast))
        for t in range(0, grad.shape[1]):
            carve_mask[indexLast][grad.shape[1] - 1 - t] = 1
            indexLast = seamMatrix[indexLast][grad.shape[1] - 1 - t][1]
            indexLast = int(round(indexLast))
    return (resized_img, resized_mask, carve_mask)


Overwriting /Users/ahmad/Desktop/CompVision/02-seam-carve/seam_carve.py


In [1]:
import numpy as np

from sys import argv, stderr
from glob import iglob
from skimage.io import imread, imsave
from pickle import load
import matplotlib.pyplot as plt
from os.path import dirname, exists
from glob import iglob

from seam_carve import seam_carve

def seam_coords(seam_mask):
    coords = np.where(seam_mask)
    t = [i for i in zip(coords[0], coords[1])]
    t.sort(key = lambda i: i[0])
    return tuple(t)

def msk_to_std(msk):
    return ((msk[:,:,0]!=0)*(-1) + (msk[:,:,1]!=0)).astype('int8')

if len(argv) != 3:
    stderr.write('Usage: %s mode input_dir\n' % argv[0])
    exit(1)

if argv[1] != '--base' and argv[1] != '--full':
    stderr.write('Usage: %s mode input_dir\n' % argv[0])
    exit(1)

mode = 'full'
input_dir = '/Users/ahmad/Desktop/CompVision/02-seam-carve/tests'
output_dir = '/Users/ahmad/Desktop/CompVision/02-seam-carve/mytests'

number = 0
TP = 0
i = 0

for filename in iglob(input_dir + '/*.png'):
    if filename.find('_mask') >= 0:
        continue    

    print(filename)

    img = imread(filename)
    if mode == 'full':
        msk = imread(filename[:-4] + '_mask.png')
        msk = msk_to_std(msk)
        number += 8
    else:
        msk = None
        number += 2

    file = open(filename[:-4] + '_seams', 'rb')
    name = "_" + repr(i) + "_"
    i += 1

    if mode == 'base':
        for orientation in ('horizontal', 'vertical'):
            kek = seam_coords(seam_carve(img, orientation + ' shrink')[2])
            res = np.zeros(img.shape)
            for pair in kek:
                res[pair[0], pair[1], 0] = 1
            fl = load(file)
            for pair in fl:
                res[pair[0], pair[1], 1] = 1
            TP += fl == kek

    elif mode == 'full':
        import timeit
        start_time = timeit.default_timer()
        # code you want to evaluate
        for m in (None, msk):
            for direction in ('shrink', 'expand'):
                for orientation in ('horizontal', 'vertical'):
                    stringMask = 'withMask'
                    if (m == None):
                        stringMask = ''
                    kek = seam_coords(seam_carve(img, orientation + ' ' + direction, mask = m)[2])
                    res = np.zeros(img.shape)
                    for pair in kek:
                        res[pair[0], pair[1], 0] = 1
                    fl = load(file)
                    for pair in fl:
                        res[pair[0], pair[1], 1] = 1
                    if (fl != kek):
                        out_filename = output_dir + '/' + name + direction + orientation + stringMask + '.png'
                        out_dirname = dirname(out_filename)
                        if not exists(out_dirname):
                            mkdir(out_dirname)
                        imsave(out_filename, res)
                    TP += fl == kek
        elapsed = timeit.default_timer() - start_time
        print(elapsed)
    file.close()

print('Accuracy: {0:.2%}'.format(TP / number))



Usage: /Users/ahmad/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py mode input_dir


/Users/ahmad/Desktop/CompVision/02-seam-carve/tests/pic_01.png


/Users/ahmad/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:76: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


6.974832774998504
/Users/ahmad/Desktop/CompVision/02-seam-carve/tests/pic_02.png
7.199068337999051
/Users/ahmad/Desktop/CompVision/02-seam-carve/tests/pic_03.png
6.363075916997332
/Users/ahmad/Desktop/CompVision/02-seam-carve/tests/pic_04.png
2.5536559769971063
/Users/ahmad/Desktop/CompVision/02-seam-carve/tests/pic_05.png
1.7695680569959222
/Users/ahmad/Desktop/CompVision/02-seam-carve/tests/pic_06.png
4.908668673997454
/Users/ahmad/Desktop/CompVision/02-seam-carve/tests/pic_07.png
3.400947302005079
Accuracy: 100.00%


In [4]:
%%writefile /Users/ahmad/Desktop/CompVision/02-seam-carve/seam_carve.py

from numpy import zeros, argmin, append, delete, insert, transpose, array, column_stack, row_stack
from math import sqrt
 
def gradient(matrix):
    resx = zeros(matrix.shape[0:2])
    resy = zeros(matrix.shape[0:2])
    for x in range(matrix.shape[0]):
        for y in range(matrix.shape[1]):
            if (x == 0):
                resx[0, y] = matrix[1, y] - matrix[0, y]
            elif (x == matrix.shape[0] - 1):
                resx[x, y] = matrix[x, y] - matrix[x - 1, y]
            else:
                resx[x, y] = matrix[x + 1, y] - matrix[x - 1, y]
 
            if (y == 0):
                resy[x, 0] = matrix[x, 1] - matrix[x, 0]
            elif (y == matrix.shape[1] - 1):
                resy[x, y] = matrix[x, y] - matrix[x, y - 1]
            else:
                resy[x, y] = matrix[x, y + 1] - matrix[x, y - 1]
 
    return (resx ** 2 + resy ** 2) ** (1 / 2)
 
def seam_carve(img, type, mask = None):
    orientation, mode = type.split()
 
    redIm = img[:, :, 0]
    greenIm = img[:, :, 1]
    blueIm = img[:, :, 2]
    bright = 0.299 * redIm + 0.587 * greenIm + 0.114 * blueIm
    brightness = gradient(bright)
 
    if mask is None:
        resized_mask = None
    else:
        # here should be anything with mask
        shape = img.shape[0] * img.shape[1]
        for x in range(mask.shape[0]):
            for y in range(mask.shape[1]):
                if (mask[x, y] > 0):
                    brightness[x, y] += shape * 256
                elif (mask[x, y] < 0):
                    brightness[x, y] -= shape * 256
 
    form = [img.shape[0], img.shape[1], 2]
    searchForMin = zeros(form)
    shape = searchForMin.shape
    null = [0, 0]
    carve_mask = zeros(img.shape[0:2])
    resized_img = zeros(null[0:2])
    resized_mask = zeros(null[0:2])
    if (orientation == 'horizontal'):
        if (mode == 'shrink'):
            for x in range(brightness.shape[0]):
                for y in range(brightness.shape[1]):
                    if (x == 0):
                        searchForMin[x, y, 0] = brightness[x, y]
                        searchForMin[x, y, 1] = -1
                    if (y == 0):
                        if (searchForMin[x - 1, y, 0] <= searchForMin[x - 1, y + 1, 0]):
                            searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x - 1, y, 0]
                            searchForMin[x, y, 1] = y
                        else:
                            searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x - 1, y + 1, 0]
                            searchForMin[x, y, 1] = y + 1
                    elif (y == brightness.shape[1] - 1):
                        if (searchForMin[x - 1, y - 1, 0] <= searchForMin[x - 1, y, 0]):
                            searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x - 1, y - 1, 0]
                            searchForMin[x, y, 1] = y - 1
                        else:
                            searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x - 1, y, 0]
                            searchForMin[x, y, 1] = y
                    else:
                        if (searchForMin[x - 1, y - 1, 0] <= searchForMin[x - 1, y, 0]):
                            if (searchForMin[x - 1, y - 1, 0] <= searchForMin[x - 1, y + 1, 0]):
                                searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x - 1, y - 1, 0]
                                searchForMin[x, y, 1] = y - 1
                            else:
                                searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x - 1, y + 1, 0]
                                searchForMin[x, y, 1] = y + 1
                        else:
                            if (searchForMin[x - 1, y, 0] <= searchForMin[x - 1, y + 1, 0]):
                                searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x - 1, y, 0]
                                searchForMin[x, y, 1] = y
                            else:
                                searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x - 1, y + 1, 0]
                                searchForMin[x, y, 1] = y + 1
 
            iter = argmin(searchForMin[shape[0] - 1, :, 0])
            carve_mask[shape[0] - 1, iter] = 1
            for x in range(shape[0] - 1, 0, -1):
                iter = int(searchForMin[x, iter, 1])
                carve_mask[x - 1, iter] = 1
        else:
            for x in range(brightness.shape[0]):
                for y in range(brightness.shape[1]):
                    if (x == 0):
                        searchForMin[x, y, 0] = brightness[x, y]
                        searchForMin[x, y, 1] = -1
                    if (y == 0):
                        if (searchForMin[x - 1, y, 0] <= searchForMin[x - 1, y + 1, 0]):
                            searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x - 1, y, 0]
                            searchForMin[x, y, 1] = y
                        else:
                            searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x - 1, y + 1, 0]
                            searchForMin[x, y, 1] = y + 1
                    elif (y == brightness.shape[1] - 1):
                        if (searchForMin[x - 1, y - 1, 0] <= searchForMin[x - 1, y, 0]):
                            searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x - 1, y - 1, 0]
                            searchForMin[x, y, 1] = y - 1
                        else:
                            searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x - 1, y, 0]
                            searchForMin[x, y, 1] = y
                    else:
                        if (searchForMin[x - 1, y - 1, 0] <= searchForMin[x - 1, y, 0]):
                            if (searchForMin[x - 1, y - 1, 0] <= searchForMin[x - 1, y + 1, 0]):
                                searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x - 1, y - 1, 0]
                                searchForMin[x, y, 1] = y - 1
                            else:
                                searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x - 1, y + 1, 0]
                                searchForMin[x, y, 1] = y + 1
                        else:
                            if (searchForMin[x - 1, y, 0] <= searchForMin[x - 1, y + 1, 0]):
                                searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x - 1, y, 0]
                                searchForMin[x, y, 1] = y
                            else:
                                searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x - 1, y + 1, 0]
                                searchForMin[x, y, 1] = y + 1
            iter = argmin(searchForMin[shape[0] - 1, :, 0])
            carve_mask[shape[0] - 1, iter] = 1
            for x in range(shape[0] - 1, 0, -1):
                iter = int(searchForMin[x, iter, 1])
                if (iter + 1 < img.shape[1]):
                    medium = img[x - 1, iter] + img[x - 1, iter + 1]
                    carve_mask[x - 1, iter] = 1
                else:
                    medium = img[x - 1, iter] + img[x - 1, iter - 1]
                    carve_mask[x - 1, iter] = 1
    else:
        if (mode == 'shrink'):
            for y in range(brightness.shape[1]):
                for x in range(brightness.shape[0]):
                    if (y == 0):
                        searchForMin[x, y, 0] = brightness[x, y]
                        searchForMin[x, y, 1] = -1
                    if (x == 0):
                        if (searchForMin[x, y - 1, 0] <= searchForMin[x + 1, y - 1, 0]):
                            searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x, y - 1, 0]
                            searchForMin[x, y, 1] = x
                        else:
                            searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x + 1, y - 1, 0]
                            searchForMin[x, y, 1] = x + 1
                    elif (x == brightness.shape[0] - 1):
                        if (searchForMin[x - 1, y - 1, 0] <= searchForMin[x, y - 1, 0]):
                            searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x - 1, y - 1, 0]
                            searchForMin[x, y, 1] = x - 1
                        else:
                            searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x, y - 1, 0]
                            searchForMin[x, y, 1] = x
                    else:
                        if (searchForMin[x - 1, y - 1, 0] <= searchForMin[x, y - 1, 0]):
                            if (searchForMin[x - 1, y - 1, 0] <= searchForMin[x + 1, y - 1, 0]):
                                searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x - 1, y - 1, 0]
                                searchForMin[x, y, 1] = x - 1
                            else:
                                searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x + 1, y - 1, 0]
                                searchForMin[x, y, 1] = x + 1
                        else:
                            if (searchForMin[x, y - 1, 0] <= searchForMin[x + 1, y - 1, 0]):
                                searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x, y - 1, 0]
                                searchForMin[x, y, 1] = x
                            else:
                                searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x + 1, y - 1, 0]
                                searchForMin[x, y, 1] = x + 1
            y = shape[1] - 1
            x = 0
            minVal = searchForMin[x, y, 0]
            iter = 0
            for x in range(1, shape[0]):
                if (searchForMin[x, y, 0] < minVal):
                    minVal = searchForMin[x, y, 0]
                    iter = x
            # transpose(resized_img)
            carve_mask[iter, shape[1] - 1] = 1
            for x in range(shape[1] - 1, 0, -1):
                iter = int(searchForMin[iter, x, 1])
                carve_mask[iter, x - 1] = 1
        else:
            for y in range(brightness.shape[1]):
                for x in range(brightness.shape[0]):
                    if (y == 0):
                        searchForMin[x, y, 0] = brightness[x, y]
                        searchForMin[x, y, 1] = -1
                    if (x == 0):
                        if (searchForMin[x, y - 1, 0] <= searchForMin[x + 1, y - 1, 0]):
                            searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x, y - 1, 0]
                            searchForMin[x, y, 1] = x
                        else:
                            searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x + 1, y - 1, 0]
                            searchForMin[x, y, 1] = x + 1
                    elif (x == brightness.shape[0] - 1):
                        if (searchForMin[x - 1, y - 1, 0] <= searchForMin[x, y - 1, 0]):
                            searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x - 1, y - 1, 0]
                            searchForMin[x, y, 1] = x - 1
                        else:
                            searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x, y - 1, 0]
                            searchForMin[x, y, 1] = x
                    else:
                        if (searchForMin[x - 1, y - 1, 0] <= searchForMin[x, y - 1, 0]):
                            if (searchForMin[x - 1, y - 1, 0] <= searchForMin[x + 1, y - 1, 0]):
                                searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x - 1, y - 1, 0]
                                searchForMin[x, y, 1] = x - 1
                            else:
                                searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x + 1, y - 1, 0]
                                searchForMin[x, y, 1] = x + 1
                        else:
                            if (searchForMin[x, y - 1, 0] <= searchForMin[x + 1, y - 1, 0]):
                                searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x, y - 1, 0]
                                searchForMin[x, y, 1] = x
                            else:
                                searchForMin[x, y, 0] = brightness[x, y] + searchForMin[x + 1, y - 1, 0]
                                searchForMin[x, y, 1] = x + 1
            y = shape[1] - 1
            x = 0
            minVal = searchForMin[x, y, 0]
            iter = 0
            for x in range(1, shape[0]):
                if (searchForMin[x, y, 0] < minVal):
                    minVal = searchForMin[x, y, 0]
                    iter = x
            carve_mask[iter, shape[1] - 1] = 1
            for x in range(shape[1] - 1, 0, -1):
                iter = int(searchForMin[iter, x, 1])
                if (iter + 1 < img.shape[0]):
                    medium = img[iter, x - 1] + img[iter + 1, x - 1]
                    carve_mask[iter, x - 1] = 1
                else:
                    medium = img[iter, x - 1] + img[iter - 1, x - 1]
                    carve_mask[iter, x - 1] = 1
    return (resized_img, resized_mask, carve_mask)

Overwriting /Users/ahmad/Desktop/CompVision/02-seam-carve/seam_carve.py


In [2]:
%%writefile /Users/ahmad/Desktop/CompVision/02-seam-carve/seam_carve.py
# %load /Users/ahmad/Desktop/02-seam-carve/seam_carve.py
import numpy as np
from numpy import zeros
 
def gradi(arr):
    gradx = zeros(arr.shape)
    grady = zeros(arr.shape)
    for i in range(0, arr.shape[0]):
        for j in range(0, arr.shape[1]):
            if (i == 0):
                gradx[i][j] = arr[i + 1][j] - arr[i][j]
            elif (i == (arr.shape[0] - 1)):
                gradx[i][j] = arr[i][j] - arr[i - 1][j]
            else:
                gradx[i][j] = arr[i + 1][j] - arr[i - 1][j]
            if (j == 0):
                grady[i][j] = arr[i][j + 1] - arr[i][j]
            elif (j == (arr.shape[1] - 1)):
                grady[i][j] = arr[i][j] - arr[i][j - 1]
            else:
                grady[i][j] = arr[i][j + 1] - arr[i][j - 1]
    return gradx, grady
 
def seam_carve(img, mode, mask=None):
    # dummy implementation — delete rightmost column of image
    resized_img = img[:, :-1, :]
    gray = 0.299 * img[..., 0]
    gray[:] += 0.587 * img[..., 1]
    gray[:] += 0.114 * img[..., 2]
    if mask is None:
        import numpy as np
        resized_mask = None
        arr = gray
        gradx, grady = gradi(arr)
        grad = np.sqrt(np.square(gradx) + np.square(grady))
 
        seamMatrix = np.zeros((grad.shape[0], grad.shape[1], 2))
        carve_mask = zeros((img.shape[0], img.shape[1]))
        if ((mode == 'horizontal shrink') or (mode == 'horizontal expand')):
            for k in range(0, grad.shape[1]):
                seamMatrix[0][k][0] = grad[0][k]
                seamMatrix[0][k][1] = 0
            for i in range(1, grad.shape[0]):
                for j in range(0, grad.shape[1]):
                    if (j == 0):
                        tempArr = np.array([seamMatrix[i - 1, j, 0], seamMatrix[i - 1, j + 1, 0]])
                        seamMatrix[i, j, 1] = j + np.argmin(tempArr)
                        seamMatrix[i][j][0] = min(tempArr) + grad[i][j]
                    elif (j == (grad.shape[1] - 1)):
                        tempArr = np.array([seamMatrix[i - 1, j - 1, 0], seamMatrix[i - 1, j, 0]])
                        seamMatrix[i, j, 1] = j - 1 + np.argmin(tempArr)
                        seamMatrix[i][j][0] = min(tempArr) + grad[i][j]
                    else:
#                         if (seamMatrix[i - 1, j - 1, 0] <= seamMatrix[i - 1, j, 0]):
#                             if (seamMatrix[i - 1, j - 1, 0] <= seamMatrix[i - 1, j + 1, 0]):
#                                 seamMatrix[i, j, 0] = grad[i, j] + seamMatrix[i - 1, j - 1, 0]
#                                 seamMatrix[i, j, 1] = j - 1
#                             else:
#                                 seamMatrix[i, j, 0] = grad[i, j] + seamMatrix[i - 1, j + 1, 0]
#                                 seamMatrix[i, j, 1] = j + 1
#                         else:
#                             if (seamMatrix[i - 1, j, 0] <= seamMatrix[i - 1, j + 1, 0]):
#                                 seamMatrix[i, j, 0] = grad[i, j] + seamMatrix[i - 1, j, 0]
#                                 seamMatrix[i, j, 1] = j
#                             else:
#                                 seamMatrix[i, j, 0] = grad[i, j] + seamMatrix[i - 1, j + 1, 0]
#                                 seamMatrix[i, j, 1] = j + 1
                        tempArr = np.array([seamMatrix[i - 1, j - 1, 0], seamMatrix[i - 1, j, 0], seamMatrix[i - 1, j + 1, 0]])
                        seamMatrix[i, j, 1] = j - 1 + np.argmin(tempArr)
                        seamMatrix[i][j][0] = min(tempArr) + grad[i][j]
            indexLast = np.argmin(seamMatrix[(grad.shape[0] - 1), :, 0])
            indexLast = int(round(indexLast))
            for t in range(0, grad.shape[0]):
                carve_mask[grad.shape[0] - 1 - t][indexLast] = 1
                indexLast = seamMatrix[grad.shape[0] - 1 - t][indexLast][1]
                indexLast = int(round(indexLast))
        else:
            for k in range(0, grad.shape[0]):
                seamMatrix[k][0][0] = grad[k][0]
                seamMatrix[k][0][1] = 0
            for j in range(1, grad.shape[1]):
                for i in range(0, grad.shape[0]):
                    if (i == 0):
                        tempArr = np.array([seamMatrix[i][j - 1][0], seamMatrix[i + 1][j - 1][0]])
                        seamMatrix[i][j][0] = min(tempArr) + grad[i][j]
                        seamMatrix[i][j][1] = (tempArr.index(min(tempArr))) + i
                    elif (i == (grad.shape[0] - 1)):
                        tempArr = np.array([seamMatrix[i - 1][j - 1][0], seamMatrix[i][j - 1][0]])
                        seamMatrix[i][j][0] = min(tempArr) + grad[i][j]
                        seamMatrix[i][j][1] = (tempArr.index(min(tempArr))) + i - 1
                    else:
#                         if (seamMatrix[i - 1, j - 1, 0] <= seamMatrix[i, j - 1, 0]):
#                             if (seamMatrix[i - 1, j - 1, 0] <= seamMatrix[i + 1, j - 1, 0]):
#                                 seamMatrix[i, j, 0] = grad[i, j] + seamMatrix[i - 1, j - 1, 0]
#                                 seamMatrix[i, j, 1] = i - 1
#                             else:
#                                 seamMatrix[i, j, 0] = grad[i, j] + seamMatrix[i + 1, j - 1, 0]
#                                 seamMatrix[i, j, 1] = i + 1
#                         else:
#                             if (seamMatrix[i , j - 1, 0] <= seamMatrix[i + 1, j - 1, 0]):
#                                 seamMatrix[i, j, 0] = grad[i, j] + seamMatrix[i, j - 1, 0]
#                                 seamMatrix[i, j, 1] = i
#                             else:
#                                 seamMatrix[i, j, 0] = grad[i, j] + seamMatrix[i + 1, j - 1, 0]
#                                 seamMatrix[i, j, 1] = i + 1
                        tempArr = np.array([seamMatrix[i - 1][j - 1][0], seamMatrix[i][j - 1][0], seamMatrix[i + 1][j - 1][0]])
                        seamMatrix[i][j][0] = min(tempArr) + grad[i][j]
                        seamMatrix[i][j][1] = (tempArr.index(min(tempArr))) + i - 1
            indexLast = np.argmin(seamMatrix[:, grad.shape[1] - 1, 0])
            indexLast = int(round(indexLast))
            for t in range(0, grad.shape[1]):
                carve_mask[indexLast][grad.shape[1] - 1 - t] = 1
                indexLast = seamMatrix[indexLast][grad.shape[1] - 1 - t][1]
                indexLast = int(round(indexLast))
    else:
        import numpy as np
        resized_mask = mask[:, :-1]
        arr = gray
        gradx, grady = gradi(arr)
        grad = np.sqrt(np.square(gradx) + np.square(grady))
        seamMatrix = np.zeros((grad.shape[0], grad.shape[1], 2))
        carve_mask = zeros(arr.shape)
        if ((mode == 'horizontal shrink') or (mode == 'horizontal expand')):
            for k in range(0, grad.shape[1]):
                seamMatrix[0][k][0] = grad[0][k]
                seamMatrix[0][k][1] = 0
            for i in range(1, grad.shape[0]):
                for j in range(0, grad.shape[1]):
                    if (j == 0):
                        tempArr = np.array([seamMatrix[i - 1, j, 0], seamMatrix[i - 1, j + 1, 0]])
                        seamMatrix[i, j, 1] = j + np.argmin(tempArr)
                        seamMatrix[i][j][0] = min(tempArr) + grad[i][j]
                        if (mask[i][j] != 0) :
                            seamMatrix[i][j][0] += mask[i][j] * 256 * seamMatrix.shape[0] * seamMatrix.shape[1]
                    elif (j == (grad.shape[1] - 1)):
                        tempArr = np.array([seamMatrix[i - 1, j - 1, 0], seamMatrix[i - 1, j, 0]])
                        seamMatrix[i, j, 1] = j - 1 + np.argmin(tempArr)
                        seamMatrix[i][j][0] = min(tempArr) + grad[i][j]
                        if (mask[i][j] != 0) :
                            seamMatrix[i][j][0] += mask[i][j] * 256 * seamMatrix.shape[0] * seamMatrix.shape[1]
                    else:
                        tempArr = np.array([seamMatrix[i - 1, j - 1, 0], seamMatrix[i - 1, j, 0], seamMatrix[i - 1, j + 1, 0]])
                        seamMatrix[i, j, 1] = j - 1 + np.argmin(tempArr)
                        seamMatrix[i][j][0] = min(tempArr) + grad[i][j]
                        if (mask[i][j] != 0) :
                            seamMatrix[i][j][0] += mask[i][j] * 256 * seamMatrix.shape[0] * seamMatrix.shape[1]
            indexLast = np.argmin(seamMatrix[(grad.shape[0] - 1), :, 0])
            indexLast = int(round(indexLast))
            for t in range(0, grad.shape[0]):
                carve_mask[grad.shape[0] - 1 - t][indexLast] = 1
                indexLast = seamMatrix[grad.shape[0] - 1 - t][indexLast][1]
                indexLast = int(round(indexLast))
        else:
            for k in range(0, grad.shape[0]):
                seamMatrix[k][0][0] = grad[k][0]
                seamMatrix[k][0][1] = 0
            for j in range(1, grad.shape[1]):
                for i in range(0, grad.shape[0]):
                    if (i == 0):
                        tempArr = np.array([seamMatrix[i][j - 1][0], seamMatrix[i + 1][j - 1][0]])
                        seamMatrix[i][j][0] = min(tempArr) + grad[i][j]
                        seamMatrix[i][j][1] = (tempArr.index(min(tempArr))) + i
                        if (mask[i][j] != 0) :
                            seamMatrix[i][j][0] += mask[i][j] * 256 * seamMatrix.shape[0] * seamMatrix.shape[1]
                    elif (i == (grad.shape[0] - 1)):
                        tempArr = np.array([seamMatrix[i - 1][j - 1][0], seamMatrix[i][j - 1][0]])
                        seamMatrix[i][j][0] = min(tempArr) + grad[i][j]
                        seamMatrix[i][j][1] = (tempArr.index(min(tempArr))) + i - 1
                        if (mask[i][j] != 0) :
                            seamMatrix[i][j][0] += mask[i][j] * 256 * seamMatrix.shape[0] * seamMatrix.shape[1]
                    else:
                        tempArr = np.array([seamMatrix[i - 1][j - 1][0], seamMatrix[i][j - 1][0], seamMatrix[i + 1][j - 1][0]])
                        seamMatrix[i][j][0] = min(tempArr) + grad[i][j]
                        seamMatrix[i][j][1] = (tempArr.index(min(tempArr))) + i - 1
                        if (mask[i][j] != 0) :
                            seamMatrix[i][j][0] += mask[i][j] * 256 * seamMatrix.shape[0] * seamMatrix.shape[1]
            indexLast = np.argmin(seamMatrix[:, grad.shape[1] - 1, 0])
            indexLast = int(round(indexLast))
            for t in range(0, grad.shape[1]):
                carve_mask[indexLast][grad.shape[1] - 1 - t] = 1
                indexLast = seamMatrix[indexLast][grad.shape[1] - 1 - t][1]
                indexLast = int(round(indexLast))
    return (resized_img, resized_mask, carve_mask)

Overwriting /Users/ahmad/Desktop/CompVision/02-seam-carve/seam_carve.py


In [3]:
int(4.999)

4

In [9]:
from numpy import zeros
import numpy as np

def gradi(arr):
    gradx = zeros(arr.shape)
    grady = zeros(arr.shape)
    for i in range(0, arr.shape[0]):
        for j in range(0, arr.shape[1]):
            if (i == 0):
                gradx[i][j] = arr[i + 1][j] - arr[i][j]
            elif (i == (arr.shape[0] - 1)):
                gradx[i][j] = arr[i][j] - arr[i - 1][j]
            else:
                gradx[i][j] = arr[i + 1][j] - arr[i - 1][j]
            if (j == 0):
                grady[i][j] = arr[i][j + 1] - arr[i][j]
            elif (j == (arr.shape[1] - 1)):
                grady[i][j] = arr[i][j] - arr[i][j - 1]
            else:
                grady[i][j] = arr[i][j + 1] - arr[i][j - 1]
    return (np.sqrt(np.square(gradx) + np.square(grady)))

arr = np.array([[1, 2, 3], [3, 4, 5], [10, 20, 30]])
gradx, grady = np.gradient(arr)
gradx *= 2
grady *= 2
for i in range(gradx.shape[1]):
    gradx[0, i] = (gradx[0, i] / 2)
    gradx[gradx.shape[0] - 1, i] = (gradx[gradx.shape[0] - 1, i] / 2)
for i in range(gradx.shape[0]):
    grady[i, 0] = (grady[i, 0] / 2)
    grady[i, gradx.shape[1] - 1] = (grady[i, gradx.shape[1] - 1] / 2)
grad = np.sqrt(np.square(gradx) + np.square(grady))
# grad = gradi(arr)
grad

array([[  2.23606798,   2.82842712,   2.23606798],
       [  9.05538514,  18.11077028,  27.01851217],
       [ 12.20655562,  25.61249695,  26.92582404]])

In [1]:
import numpy as np

from sys import argv, stderr
from glob import iglob
from skimage.io import imread
from pickle import load

from seam_carve import seam_carve

def seam_coords(seam_mask):
    coords = np.where(seam_mask)
    t = [i for i in zip(coords[0], coords[1])]
    t.sort(key = lambda i: i[0])
    return tuple(t)

def msk_to_std(msk):
    return ((msk[:,:,0]!=0)*(-1) + (msk[:,:,1]!=0)).astype('int8')

if len(argv) != 3:
    stderr.write('Usage: %s mode input_dir\n' % argv[0])
    exit(1)

if argv[1] != '--base' and argv[1] != '--full':
    stderr.write('Usage: %s mode input_dir\n' % argv[0])
    exit(1)

mode = 'full'
input_dir = '/Users/ahmad/Desktop/CompVision/02-seam-carve/tests'
output_dir = '/Users/ahmad/Desktop/CompVision/02-seam-carve/mytests'

number = 0
TP = 0
for filename in iglob(input_dir + '/*.png'):
    import timeit
    start_time = timeit.default_timer()
    # code you want to evaluate
    if filename.find('_mask') >= 0:
        continue    

    print(filename)

    img = imread(filename)
    if mode == 'full':
        msk = imread(filename[:-4] + '_mask.png')
        msk = msk_to_std(msk)
        number += 8
    else:
        msk = None
        number += 2

    file = open(filename[:-4] + '_seams', 'rb')

    if mode == 'base':
        for orientation in ('horizontal', 'vertical'):
            TP += load(file) == seam_coords(seam_carve(img, orientation + ' shrink')[2])
    elif mode == 'full':
        for m in (None, msk):
            for direction in ('shrink', 'expand'):
                for orientation in ('horizontal', 'vertical'):
                    TP += load(file) == seam_coords(seam_carve(img, orientation + ' ' + direction, mask = m)[2])
        elapsed = timeit.default_timer() - start_time
        print(elapsed)
    file.close()

print('Accuracy: {0:.2%}'.format(TP / number))


Usage: /Users/ahmad/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py mode input_dir


/Users/ahmad/Desktop/CompVision/02-seam-carve/tests/pic_01.png
15.178242597001372
/Users/ahmad/Desktop/CompVision/02-seam-carve/tests/pic_02.png
16.02173409399984
/Users/ahmad/Desktop/CompVision/02-seam-carve/tests/pic_03.png
14.787713182995503
/Users/ahmad/Desktop/CompVision/02-seam-carve/tests/pic_04.png
6.130725805000111
/Users/ahmad/Desktop/CompVision/02-seam-carve/tests/pic_05.png
4.259853016999841
/Users/ahmad/Desktop/CompVision/02-seam-carve/tests/pic_06.png
11.267552824996528
/Users/ahmad/Desktop/CompVision/02-seam-carve/tests/pic_07.png
8.168536502002098
Accuracy: 100.00%
